In [1]:
from models import *
from utils_function import *
from test_function import *

import math
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import itertools

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler


In [2]:
X, y = LBNL59()
X_columns_to_normalize = X.columns.difference(['date'])
X_scaler = MinMaxScaler()
X[X_columns_to_normalize] = X_scaler.fit_transform(X[X_columns_to_normalize])
y_columns_to_sum = y.columns.difference(['date'])
y['sum'] = y[y_columns_to_sum].sum(axis=1)
y = y[["date", 'sum']]
X = X[X["date"] >= '2018-09-16']
y = y[y["date"] >= '2018-09-16']

X_train = X[X["date"] <= '2020-10-16']
y_train = y[y["date"] <= '2020-10-16']

X_test = X[X["date"] >= '2020-10-16']
y_test = y[y["date"] >= '2020-10-16']

In [3]:
seq_length = 50
predict_length = 1
batch_size = 100

Train_dataset = TimeSeriesDataset_sep(X_train, y_train, seq_length, predict_length = predict_length)
Train_dataloader = DataLoader(Train_dataset, batch_size=batch_size, shuffle=False)

Test_dataset = TimeSeriesDataset_sep(X_test, y_test, seq_length, predict_length = predict_length)
Test_dataloader = DataLoader(Test_dataset, batch_size=batch_size, shuffle=False)


In [4]:
num_epochs = 50
input_size = 306
output_size = 1
hidden_size = 250
num_layers = 2
num_heads = 10
model = BiLSTMTransformer(input_size, hidden_size, num_layers, output_size, num_heads, predict_length).to(device)

In [5]:
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
teacher_forcing_ratio = 0
criterion = nn.MSELoss()
#criterion = nn.SmoothL1Loss()  # 用于回归任务
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
loss_per_epoch = []
val_mse_per_epoch = []
val_r2_per_epoch = []

for epoch in range(num_epochs):
    model.train()  # 确保模型在训练模式下
    for external, internal, batch_y in Train_dataloader:
        external, internal, batch_y = external.to(device), internal.to(device), batch_y.to(device)

        # 前向传播
        total_loss = 0
        y = internal[:, -1:, :]
        for step in range(predict_length):
            outputs = model(external, internal, y)
            next_pred = outputs[:, -1:, :]
            #outputs = model(batch_X, batch_y.view(batch_y.shape[0], batch_y.shape[2]))
            loss = criterion(next_pred, batch_y[:, step:step+1, :])
            total_loss += loss
            if np.random.rand() < teacher_forcing_ratio:
                next_input = batch_y[:, step:step + 1, :]
            else:
                next_input = next_pred
            y = torch.cat([y, next_input], dim=1)
                
        total_loss = total_loss / predict_length
        # 反向传播和优化
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

    #scheduler.step()

    loss_per_epoch.append(loss.item())
    teacher_forcing_ratio -= 0.05

    # 评估验证集
    val_loss, (val_mse, val_mae, val_r2, val_mape) = evaluate_Transformer(model, Test_dataloader, criterion, device, [mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error])
    val_mse_per_epoch.append(val_mse)
    val_r2_per_epoch.append(val_r2)

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}, Val MSE: {val_mse:.4f}, Val MAE: {val_mae:.4f}, Val R²: {val_r2:.4f}, Val MAPE: {val_mape:.4f}')


Epoch [1/50], Loss: 1991.8296, Val Loss: 323.7746, Val MSE: 324.1068, Val MAE: 15.6812, Val R²: -1.0721, Val MAPE: 0.5207
Epoch [2/50], Loss: 1300.6281, Val Loss: 170.8737, Val MSE: 172.2729, Val MAE: 11.6555, Val R²: -0.1014, Val MAPE: 0.5561
Epoch [3/50], Loss: 813.9973, Val Loss: 171.0255, Val MSE: 173.3895, Val MAE: 9.4908, Val R²: -0.1085, Val MAPE: 0.6429
Epoch [4/50], Loss: 479.6570, Val Loss: 26.3894, Val MSE: 26.6823, Val MAE: 3.2613, Val R²: 0.8294, Val MAPE: 0.1583
Epoch [5/50], Loss: 272.1000, Val Loss: 18.8804, Val MSE: 19.0759, Val MAE: 2.8538, Val R²: 0.8780, Val MAPE: 0.1472
Epoch [6/50], Loss: 154.6667, Val Loss: 18.5413, Val MSE: 18.6672, Val MAE: 3.0937, Val R²: 0.8807, Val MAPE: 0.1356
Epoch [7/50], Loss: 107.2104, Val Loss: 17.9840, Val MSE: 18.1430, Val MAE: 2.9674, Val R²: 0.8840, Val MAPE: 0.1430
Epoch [8/50], Loss: 56.7616, Val Loss: 15.0907, Val MSE: 15.2345, Val MAE: 2.5865, Val R²: 0.9026, Val MAPE: 0.1307
Epoch [9/50], Loss: 45.4315, Val Loss: 14.8285, Val 